In [1]:
%cd ..

d:\My Projects\Toxic Tagger


In [13]:
import os
import mlflow
import dagshub
from dotenv import load_dotenv
load_dotenv()

repo_owner = os.getenv("OWNER")
repo_name = os.getenv("REPO")
uri = os.getenv("MLFLOW_URI")

mlflow.set_tracking_uri(uri)

if repo_owner is None:
	raise EnvironmentError("Missing dagshub logging environment credentials.")
dagshub.init(repo_owner=repo_owner, repo_name=repo_name, mlflow=True)

Accessing as SubinoyBera

Initialized MLflow to track repo "SubinoyBera/Toxic-TweetTagger"

Repository SubinoyBera/Toxic-TweetTagger initialized!

In [4]:
import pandas as pd

In [14]:
model_name = "ToxicTagger-Models"
stage = "Production"
model_uri = f"models:/{model_name}/{stage}"
new_model = mlflow.pyfunc.load_model(model_uri)

c:\Users\subin\.conda\envs\ml-dl.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
text = ["i love to fuck u very much sweetie"]
df = pd.DataFrame(text, columns=["comments"])

response = new_model.predict(df)

In [19]:
label = int(response["class_label"][0])

In [20]:
label

1

In [2]:
import joblib
with open("serve_api/model/python_model.pkl", "rb") as f:
    model = joblib.load(f)

In [ ]:
type(model.model).__name__

'XGBClassifier'

In [11]:
type(model.vectorizer).__name__

'TfidfVectorizer'

In [ ]:
import pandas as pd
text = ["i love god very much"]
df = pd.DataFrame(text, columns=["comments"])

response = model.predict(context=None, model_input=df)

In [12]:
label = int(response["class_label"][0])
label

0

### Testing MongoDB Connection

In [1]:
import pymongo
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
DB_NAME = "Toxic_Tweets"
COLLECTION_NAME = "toxic_tweet_data"
CONNECTION_URL = os.getenv("MONGO_CONNECTION_URL")
MONGO_USERNAME = os.getenv("MONGO_USERNAME")
MONGO_PASSWORD = os.getenv("MONGO_PASSWORD")
MONGO_URL = os.getenv("MONGO_URL")

In [24]:
from urllib.parse import quote_plus
import pymongo
import certifi

ca = certifi.where()

username = quote_plus(str(MONGO_USERNAME).strip())
password = quote_plus(str(MONGO_PASSWORD).strip())

CONNECTION_URL = f"mongodb+srv://{username}:{password}@{MONGO_URL}"

# Create SSL context forcing TLS 1.2
client = pymongo.MongoClient(CONNECTION_URL, tlsCAFile=ca)

print(client.server_info())

db = client[DB_NAME]
collection = db[COLLECTION_NAME]

df = pd.DataFrame(list(collection.find()))

{'version': '8.0.12', 'gitVersion': 'b60fc6875b5fb4b63cc0dbbd8dda0d6d6277921a', 'modules': ['enterprise'], 'allocator': 'tcmalloc-google', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [8, 0, 12, 0], 'bits': 64, 'debug': False, 'maxBsonObjectSize': 16777216, 'storageEngines': ['devnull', 'inMemory', 'queryable_wt', 'wiredTiger'], 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1754987630, 40), 'signature': {'hash': b"\x13\x8b\x072Qx,\xf5\xe1\xe5{\xebk\xe1o,\x99\xa6'\xac", 'keyId': 7479309989748670466}}, 'operationTime': Timestamp(1754987630, 40)}


In [25]:
df.head()

,_id,Content,Label
0,68982ace71f1b2ad73f4d654,retweet to the rejects who constantly call my ...,1
1,68982ace71f1b2ad73f4d655,i purpose that whatever attack everyone who ca...,1
2,68982ace71f1b2ad73f4d656,are you fucking kidding me you deserve to fuck...,1
3,68982ace71f1b2ad73f4d657,retweet i am not racist and i am not sexist i ...,1
4,68982ace71f1b2ad73f4d658,just imagine if the two million spent on the s...,1


In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120000 entries, 0 to 119999
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   _id      120000 non-null  object
 1   Content  120000 non-null  object
 2   Label    120000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 2.7+ MB
